In [1]:
import numpy as np
from scikits.odes.ode import ode

In [2]:
#Import local functions
sys.path.append('../functions')
from functions import nonlinearity, repression, activation

In [ ]:
#OLD CODE TRYING TO AUTOCOMPLETE SYSTEM OF DES
#Specify model parameters 
X = 2 #Number of xs in model
E = 2 #Number of enzymes in model

#Initialize weights
W = np.random.normal(0, 0.1, [E, 3]) #K, theta, n are 3 dimensions
V = np.random.normal(0, 0.1, [E, 3]) #same but for activation
t = np.linspace(0, 5e4, 101)

#Constants
Vin = 1
e0 = 0.0467 
lam = 1.93E-4 #1/s
#Assume equal kinetics for all three enzymes - CHANGE THIS LATER
kc = 12 #1/s
km = 10 

#Establish stoichiometric matrix
S = np.array([[-1, 0], [1, -1]])
#Establish input matrix
I = np.zeros(X)
I[0] = Vin
#Specify which Xs can act as transcription factors
T = 1
W = W[T] #Filter W so only fitting parameters of possible transcriptional controls
V = V[T]
#Establish system of differential equations
y = np.array([2290.,0.,1.,0.])
ydot = []
#Kinetic equations
for i in range(X):
    ydot = I[i] - lam*y[i] + S*nonlinearity(y[X:], kc, km)
j = 0
for i in range(X,X+E):
    ydot = -lam*y[i] + repression(y[T], W[j, 0], W[j, 1], W[j, 2]) + activation(y[T], V[j, 0], V[j, 1], V[j, 2])
    j += 1
#Add J1, J2
ydot = (I[0] + S*nonlinearity(y[1], kc, km))**2

In [18]:
#Default information - user can change but mostly won't
#Constants (set to default)
kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467

#Integration conditions
y = np.array([2290.,0.,1.,0.])
t = np.linspace(0, 5e4, 100) 

In [132]:
#User specified information
X = 2 #Number of chemical species in pathway
E = 2 #Number of enzymes in pathway

S = np.array([[-1, 0], [1, -1]]) #Stoichiometric matrix defining chemical reactions

I = np.zeros(X) #Input matrix defining inputs to pathway
I[0] = Vin #Inputs to pathway

T = [1] #Xs that can act as TFs

In [133]:
#Genetic parameter values for circuit
W = np.zeros([len(T), E, 3]) #K, theta, n are 3 dimensions

#Manually set initial conditions
for t in range(len(T)):
    for e in range(E):
        W[t][e] = [2., 0.1,1E-4]

In [139]:
#Architecture matrix
A = np.zeros([len(T), E, 2]) #3 methods of action at each locus for each TF

In [149]:
#Try to replicate dual control
A[0][0] = [0, 1]
A[0][1] = [1, 0]

In [159]:
ydot = []
for i in range(X): #Kinetic equations
    ydot.append(I[i] - lam*y[i] + np.sum(S[i]*nonlinearity(y[X:], kc, km)))
# for e in range(E): #Genetic equations
#     ydot.append(-lam*y[i+X] + np.sum([np.sum(A[t][e]*[activation(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), repression(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0])]) for t in range(len(T))]))
# ydot.append((I[0] - y[X]*nonlinearity(y[X-1], kc, km))**2) #J1
# ydot.append(np.sum([np.sum([np.sum(A[t][e]*[activation(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), repression(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0])]) for t in range(len(T))]) for e in range(E)]))#J2
ydot = np.array(ydot)
ydot

array([-0.53287909,  1.09090909])

In [151]:
def dual_control_sundials(t, y, ydot):
        kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - lam*y[1]
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
        ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = repression(y[1], k1, theta1, n1) + activation(y[1], k2, theta2, n2)

n1, n2, theta1, theta2, k1, k2 = [2.,2., 0.1, 0.1, 1E-4, 1E-4]
ydot_dc = np.zeros(6)
dual_control_sundials(t, y, ydot_dc)
ydot_dc

array([ 6.65217391e-05,  1.19478261e+01, -9.30000000e-05,  0.00000000e+00,
        1.00000000e+00,  1.00000000e-04])